In [3]:
import os, base64

directory="images"
index="000000"

image_data = []
for folder in os.listdir(directory):
    if not folder.startswith("cam"): continue
    for filename in os.listdir(f"{directory}/{folder}"):
        if filename.startswith(index) and "Distance" not in filename:
            with open(f"{directory}/{folder}/{filename}", 'rb') as f:
                image_data.append(base64.b64encode(f.read()).decode('utf-8'))

In [30]:
folders = list()
for folder in os.listdir('images'):
    if os.path.isdir(f"images/{folder}"):
        folders.append(folder)
folders

['CoalMineExposure', 'DesertGasStation']

In [ ]:
import os
from dsta_mvs.dsta_mvs.mvs_utils.metadata_reader import MetadataReader

directory="images"
metadata = MetadataReader(directory)
metadata.read_metadata_and_initialize_dirs(f"{directory}/metadata.json", f"{directory}/frame_graph.json")

In [5]:
from dsta_mvs.dsta_mvs.mvs_utils.image_io.image_read import read_compressed_float

print(read_compressed_float("images/CoalMineExposure/Pose_000/cam0/000000_FisheyeDistance.png"))

0